In [1]:
%load_ext autoreload
%autoreload 2

In [60]:
#project
from macroecon_wrappy import (
    Auth,
    Metric,
    Measure,
    Epoch,
    plots
)
from macroecon_wrappy.adapters import (
    FredApi,
    YahooFin
)
from macroecon_wrappy.extractors import (
    NberExtract
)

#external
from fredapi import Fred
import yfinance as yf

#system
from pathlib import Path

In [29]:
secrets_path = Path('/workspaces/API-macroecon-wrappy/SECRETS.yaml')
cache_path = Path('/workspaces/API-macroecon-wrappy/tests/tmp/')
auth = Auth(secrets_path, cache_path)
auth.load_secrets()

True

In [61]:
#fred
FredApi.set_wrapper(auth, Fred)
gdp = FredApi.get_data('GDP')
gdi = FredApi.get_data('GDI')
pop = FredApi.get_data('POPTHM')

In [31]:
#nber
NberExtract.set_config(auth)
name = 'recessions'
recessions = NberExtract.get_data(name)

first two columns will be renamed: ['start', 'end']


In [4]:
#yahoo
YahooFin.set_wrapper(auth, yf)
msft = YahooFin.get_data(tickers='MSFT')

[*********************100%%**********************]  1 of 1 completed


In [62]:
output = Measure([gdp, gdi, pop], recessions)

In [63]:
assert output.get_cycle().df().shape == (35,2)
assert isinstance(output, Measure) == True
gdp._metadata['units'], gdi._metadata['units'], pop._metadata['units']

('Billions of Dollars', 'Billions of Dollars', 'Thousands')

In [66]:
from macroecon_wrappy import plots
plots.graph_ts_js(output, cols=[], recession_bars=True, log_scale='y')

alt.VConcatChart(...)